# testAPI

Created by Davy Yue 2017-06-22


### Imports

In [18]:
import itertools
import string
import os

import numpy as np

from msibi import MSIBI, State, Pair, mie
import mdtraj as md

'1.8.0'

### Actual Code

Where the real magic happens

In [21]:
t = md.load('traj_unwrapped.dcd', top='start_aa.hoomdxml')
print(t)
print("Number of Chains: {0}".format(t.n_chains))
print("Topology: {0}".format(t.top))

<mdtraj.Trajectory with 500 frames, 3072 atoms, 1024 residues, and unitcells>
Number of Chains: 1024
Topology: <mdtraj.Topology with 1024 chains, 1024 residues, 3072 atoms, 2048 bonds>
Topology 2: <mdtraj.Topology with 1024 chains, 1024 residues, 3072 atoms, 2048 bonds>
